## IDR comparison tool

### Part 1: Running a query
* To get started, execute the cell below. This will initialize a graphical user interface for the comparison tool.
* After it runs, you should see a play button with the label `Initialize IDR Analysis Tool`; click it to continue
* You will see fields to (1) enter a COG ID to query, (2) select the datasets you want to compare, and (3) select the IDR properties you want to compare
* Once you have made your selections, click the second play button labeled `Run query and plot`

In [3]:
import os, sys, ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib

SCRIPT_DIR = os.path.abspath("python-scripts")
if SCRIPT_DIR not in sys.path:
    sys.path.append(SCRIPT_DIR)

import idr_analysis_setup
importlib.reload(idr_analysis_setup)

init_button = widgets.Button(
    description="▶ Initialize IDR Analysis Tool",
    button_style="primary",
    layout=widgets.Layout(width="280px")
)

init_output = widgets.Output()

def on_init_clicked(b):
    with init_output:
        clear_output()
        ui = idr_analysis_setup.create_idr_ui()
        display(ui)

init_button.on_click(on_init_clicked)

display(HTML("<h3>IDR Analysis Tool</h3>"))
display(init_button, init_output)

Button(button_style='primary', description='▶ Initialize IDR Analysis Tool', layout=Layout(width='280px'), sty…

Output()

### Part 2: Saving the results
* After you have run at least one query, you can execute the below code cell to create a user interface for saving the results
* Once you have chosen a file name, click "Save results as CSV"
* Note well, if you have not yet run a query or loaded the underlying code by running the code cell in Part 1, this will give you an error

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink

save_label = widgets.HTML(
    "<b>Save results:</b> After running an analysis above, "
    "save the latest results table as a CSV file."
)

filename_input = widgets.Text(
    description="Filename:",
    value="idr_query_results.csv",
    layout=widgets.Layout(width="320px")
)

save_button = widgets.Button(
    description="💾 Save results as CSV",
    button_style="",
    layout=widgets.Layout(width="220px")
)

save_output = widgets.Output()

def on_save_clicked(b):
    with save_output:
        clear_output()
        df = idr_analysis_setup.LAST_RESULTS

        if df is None:
            print("No results available. Please run an analysis first.")
            return
        if df.empty:
            print("The last results table is empty; nothing to save.")
            return

        fname = (filename_input.value or "idr_query_results.csv").strip()

        # keep it simple / safe
        if "/" in fname or "\\" in fname:
            print("Please use a simple filename without path separators.")
            return

        df.to_csv(fname, index=False)
        print(f"Saved results to '{fname}' in the current directory.")
        try:
            display(FileLink(fname))
        except Exception:
            pass  # if FileLink not available, they can grab it from the file browser

save_button.on_click(on_save_clicked)

display(widgets.VBox([
    save_label,
    widgets.HBox([filename_input, save_button]),
    save_output
]))